In [2]:
# import libraries
from statsbombpy import sb
import json
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer.pitch import Pitch
import seaborn as sns

In [3]:
# obtain list of matches for specific cometition and season 
df_matches = sb.matches(competition_id=55, season_id=43)
df_matches

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3788753,2021-06-16,15:00:00.000,Europe - UEFA Euro,2020,Finland,Russia,0,1,available,...,2021-09-22T16:39:05.697512,2,Group Stage,Saint-Petersburg Stadium,Danny Desmond Makkelie,Markku Kanerva,Stanislav Cherchesov,1.1.0,2,2
1,3788765,2021-06-20,18:00:00.000,Europe - UEFA Euro,2020,Switzerland,Turkey,3,1,available,...,2021-11-11T13:54:37.507376,3,Group Stage,Bakı Olimpiya Stadionu,Slavko Vinčić,Vladimir Petković,Şenol Güneş,1.1.0,2,2
2,3795107,2021-07-02,21:00:00.000,Europe - UEFA Euro,2020,Belgium,Italy,1,2,available,...,2021-09-23T00:02:51.495862,5,Quarter-finals,Allianz Arena,Slavko Vinčić,Roberto Martínez Montoliú,Roberto Mancini,1.1.0,2,2
3,3795221,2021-07-07,21:00:00.000,Europe - UEFA Euro,2020,England,Denmark,2,1,available,...,2021-09-22T22:33:37.494366,6,Semi-finals,Wembley Stadium,Danny Desmond Makkelie,Gareth Southgate,Kasper Hjulmand,1.1.0,2,2
4,3795506,2021-07-11,21:00:00.000,Europe - UEFA Euro,2020,Italy,England,1,1,available,...,2021-09-22T22:40:31.690550,7,Final,Wembley Stadium,Björn Kuipers,Roberto Mancini,Gareth Southgate,1.1.0,2,2
5,3794688,2021-06-29,18:00:00.000,Europe - UEFA Euro,2020,England,Germany,2,0,available,...,2021-09-22T23:46:25.457149,4,Round of 16,Wembley Stadium,Danny Desmond Makkelie,Gareth Southgate,Joachim Löw,1.1.0,2,2
6,3794692,2021-06-29,21:00:00.000,Europe - UEFA Euro,2020,Sweden,Ukraine,1,2,available,...,2021-09-22T23:38:19.483199,4,Round of 16,Hampden Park,Daniele Orsato,Jan Olof Andersson,Andrii Shevchenko,1.1.0,2,2
7,3794686,2021-06-28,18:00:00.000,Europe - UEFA Euro,2020,Croatia,Spain,3,5,available,...,2021-09-22T23:30:10.428563,4,Round of 16,Parken,Cüneyt Çakιr,Zlatko Dalić,Luis Enrique Martínez García,1.1.0,2,2
8,3794687,2021-06-27,21:00:00.000,Europe - UEFA Euro,2020,Belgium,Portugal,1,0,available,...,2021-09-22T16:40:48.020474,4,Round of 16,Estadio de La Cartuja,Felix Brych,Roberto Martínez Montoliú,Fernando Manuel Fernandes da Costa Santos,1.1.0,2,2
9,3794685,2021-06-26,21:00:00.000,Europe - UEFA Euro,2020,Italy,Austria,2,1,available,...,2021-09-22T16:40:41.310001,4,Round of 16,Wembley Stadium,Anthony Taylor,Roberto Mancini,Franco Foda,1.1.0,2,2


In [5]:
# obtain event info for specific match
df_events = sb.events(match_id = 3788741)
df_events

,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,...,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,timestamp,type,under_pressure
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"{'formation': 4141, 'lineup': [{'player': {'id...",Turkey,00:00:00.000,Starting XI,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"{'formation': 433, 'lineup': [{'player': {'id'...",Italy,00:00:00.000,Starting XI,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:00.000,Half Start,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Turkey,00:00:00.000,Half Start,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:00.000,Half Start,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:04:29.312,Injury Stoppage,NaN
3799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Turkey,00:04:31.909,Injury Stoppage,NaN
3800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:38:59.616,Injury Stoppage,NaN
3801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Turkey,00:07:12.643,Own Goal Against,NaN


In [6]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3803 entries, 0 to 3802
Data columns (total 84 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ball_receipt_outcome            137 non-null    object 
 1   ball_recovery_offensive         1 non-null      object 
 2   ball_recovery_recovery_failure  6 non-null      object 
 3   block_deflection                1 non-null      object 
 4   carry_end_location              862 non-null    object 
 5   clearance_aerial_won            14 non-null     object 
 6   clearance_body_part             39 non-null     object 
 7   clearance_head                  25 non-null     object 
 8   clearance_left_foot             8 non-null      object 
 9   clearance_right_foot            6 non-null      object 
 10  counterpress                    127 non-null    object 
 11  dribble_nutmeg                  1 non-null      object 
 12  dribble_outcome                 32

In [7]:
df_throwIn = df_events.loc[df_events['play_pattern'] == 'From Throw In']
df_throwIn

,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,...,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,timestamp,type,under_pressure
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:12.163,Pass,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:16.420,Pass,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:20.025,Pass,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:24.290,Pass,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:27.721,Pass,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:39:42.798,Foul Won,True
3769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:14:33.536,Dispossessed,True
3770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:20:56.637,Dispossessed,True
3772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Turkey,00:02:08.338,Dispossessed,True


In [8]:
df_throwIn_sort = df_throwIn.sort_values(by = "index")
df_throwIn_sort

,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,...,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,timestamp,type,under_pressure
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:12.163,Pass,NaN
1067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:14.093,Ball Receipt*,NaN
2088,NaN,NaN,NaN,NaN,"[3.3, 12.7]",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:14.093,Carry,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:16.420,Pass,NaN
1068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:18.242,Ball Receipt*,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:46:42.050,Ball Receipt*,NaN
2935,NaN,NaN,NaN,NaN,"[33.9, 49.2]",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:46:42.050,Carry,True
3320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Turkey,00:46:42.655,Pressure,NaN
1050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:46:42.815,Pass,True


In [9]:
df_throwIn_final = df_throwIn_sort.reset_index()
df_throwIn_final

,level_0,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,timestamp,type,under_pressure
0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:12.163,Pass,NaN
1,1067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:14.093,Ball Receipt*,NaN
2,2088,NaN,NaN,NaN,NaN,"[3.3, 12.7]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:14.093,Carry,NaN
3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:16.420,Pass,NaN
4,1068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:18.242,Ball Receipt*,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,2070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:46:42.050,Ball Receipt*,NaN
1165,2935,NaN,NaN,NaN,NaN,"[33.9, 49.2]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:46:42.050,Carry,True
1166,3320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Turkey,00:46:42.655,Pressure,NaN
1167,1050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:46:42.815,Pass,True


In [10]:
# create dictionary deeming each throw in a success or failure
# success defined as retaining possession for at least 7 seconds 

from datetime import datetime

throw_in_outcomes = {}

current_possession = 0
looking_for_result = False

for row in df_throwIn_final.index:
    if df_throwIn_final["pass_type"][row] == "Throw-in" and looking_for_result == False:
        initial_time = df_throwIn_final["timestamp"][row]
        initial_dt_time = datetime.strptime(initial_time, '%H:%M:%S.%f')
        current_possession = df_throwIn_final["possession"][row]
        looking_for_result = True

        # print(current_possession)
    # print((df_throwIn_final["possession"][row]) > current_possession)
    # print(current_possession)
    # print(df_throwIn_final["possession"][row])
    # print("-")


    elif (looking_for_result == True) and ((df_throwIn_final["possession"][row]) > current_possession):
        end_time = df_throwIn_final["timestamp"][row]
        end_dt_time = datetime.strptime(end_time, '%H:%M:%S.%f')
        time_diff = end_dt_time - initial_dt_time
        if time_diff.total_seconds() >= 7:
            throw_in_outcomes[current_possession] = "Success"
            looking_for_result = False
        else:
            throw_in_outcomes[current_possession] = "Failure"
            looking_for_result = False
        if df_throwIn_final["pass_type"][row] == "Throw-in":
            initial_time = df_throwIn_final["timestamp"][row]
            initial_dt_time = datetime.strptime(initial_time, '%H:%M:%S.%f')
            current_possession = df_throwIn_final["possession"][row]
            looking_for_result = True

print(throw_in_outcomes)    

{3: 'Success', 4: 'Success', 5: 'Success', 7: 'Success', 8: 'Success', 10: 'Success', 12: 'Success', 13: 'Success', 14: 'Success', 17: 'Success', 18: 'Success', 23: 'Success', 33: 'Success', 34: 'Success', 38: 'Success', 41: 'Success', 42: 'Success', 46: 'Success', 47: 'Success', 52: 'Success', 56: 'Success', 62: 'Success', 64: 'Success', 71: 'Success', 75: 'Success', 80: 'Success', 98: 'Success', 99: 'Success', 104: 'Failure', 115: 'Success', 116: 'Success', 129: 'Success', 133: 'Success', 137: 'Success', 144: 'Success', 145: 'Success', 147: 'Success', 151: 'Success', 156: 'Success', 165: 'Success', 166: 'Success', 167: 'Success', 168: 'Success', 170: 'Success'}


In [16]:
# create dictionary deeming each throw in a success or failure
# success defined as retaining possession for at least 7 seconds 

from datetime import datetime

throw_in_outcomes = {}

current_possession = 0
i = 0

elapsed_time = 0
looking_for_result = False
throwin_row = 0
throwin_count = 0

while i < len(df_throwIn_final):
    if (df_throwIn_final.loc[i, "pass_type"] == "Throw-in") and (looking_for_result == False):
        throwin_count += 1
        throwin_row = i
        elapsed_time = 0
        start_time = df_throwIn_final.loc[i, "timestamp"]
        start_dt_time = datetime.strptime(start_time, '%H:%M:%S.%f')
        current_possession = df_throwIn_final.loc[i, "possession"]
        looking_for_result = True
        print(throwin_row)
    
    elif looking_for_result == True:
        end_time = df_throwIn_final.loc[i, "timestamp"]
        end_dt_time = datetime.strptime(end_time, '%H:%M:%S.%f')
        elapsed_time = elapsed_time + (end_dt_time - start_dt_time).total_seconds()
        start_dt_time = end_dt_time

        if elapsed_time >= 7:
            if df_throwIn_final.loc[i, "possession"] == current_possession:
                throw_in_outcomes[throwin_count] = "Success"
            else:
                throw_in_outcomes[throwin_count] = "Failure"
                i = throwin_row
            looking_for_result = False
    
    i = i + 1

throw_in_outcomes


0
22
26
33
42
78
89
107
199
207
232
240
249
264
280
356
388
413
447
456
460
462
473
498
517
541
585
605
642
776
787
793
801
849
868
878
934
941
1018
1035
1068
1087
1102
1108
1113


{1: 'Success',
 2: 'Failure',
 3: 'Failure',
 4: 'Success',
 5: 'Success',
 6: 'Success',
 7: 'Success',
 8: 'Success',
 9: 'Failure',
 10: 'Success',
 11: 'Failure',
 12: 'Failure',
 13: 'Success',
 14: 'Success',
 15: 'Success',
 16: 'Success',
 17: 'Success',
 18: 'Success',
 19: 'Failure',
 20: 'Failure',
 21: 'Failure',
 22: 'Success',
 23: 'Success',
 24: 'Success',
 25: 'Success',
 26: 'Success',
 27: 'Success',
 28: 'Success',
 29: 'Success',
 30: 'Success',
 31: 'Failure',
 32: 'Success',
 33: 'Success',
 34: 'Success',
 35: 'Failure',
 36: 'Success',
 37: 'Failure',
 38: 'Success',
 39: 'Success',
 40: 'Success',
 41: 'Success',
 42: 'Success',
 43: 'Failure',
 44: 'Failure',
 45: 'Success'}